In [2]:
import sys

print(sys.version)

3.7.10 (default, Feb 26 2021, 13:06:18) [MSC v.1916 64 bit (AMD64)]


# 1. 코루틴

- 제너레이터를 이용해서 처리
- 실제 값을 yield 를 통해 입력값을 받고 처리함

In [57]:
from collections.abc import Generator, Coroutine

## 간단한 코루틴

- yield를 표현식으로 사용가능

In [1]:
def co():
    while True:
        x = yield 12
        print(x)

In [2]:
corou = co()

In [3]:
corou

<generator object co at 0x000001521AE9AB48>

In [4]:
next(corou)

12

In [5]:
corou.send(123)

123


12

In [59]:
isinstance(corou, Generator)

True

In [60]:
isinstance(corou, Coroutine)

False

### 실제 값을 전달해서만 처리하는 코루틴 작성

In [6]:
def co__():
    while True:
        x = yield 
        print(x)

In [7]:
corou__ = co__()

In [8]:
next(corou__)

In [9]:
corou__.send(123)

123


In [61]:
isinstance(corou__, Coroutine)

False

## 코루틴을 위한 Generator 메소드 정의

-  send

- throw

- close

In [67]:
from collections.abc import Iterator, Generator

In [68]:
set(dir(Generator)) - set(dir(Iterator))

{'close', 'send', 'throw'}

##  send() 이해하기

In [69]:
help(Generator.send)

Help on function send in module collections.abc:

send(self, value)
    Send a value into the generator.
    Return next yielded value or raise StopIteration.



### 제너레이터로 코루틴 정의

In [15]:
def gener():
    x = yield 1
    print(f" 1 x = {x}")
    y = yield x+1
    print(f" 2 x = {x}")
    z = yield y+1
    print(f" 3 y = {y}")

In [16]:
gen = gener()

In [17]:
gen

<generator object gener at 0x000001521AE9ACC8>

## 코루틴 실행순서 알아보기

### 코루틴 작동

In [18]:
next(gen)

1

### 코루틴 정보 전달

In [19]:
gen.send(2)

 1 x = 2


3

### 코루틴 정보 전달

In [20]:
gen.send(3)

 2 x = 2


4

### 코루틴 정보 전달

In [21]:
gen.send(4)

 3 y = 3


StopIteration: 

## 실습 : 코루틴 통해 연속적인 수를 더한다.

1. 제너레이터 안에 보낸 값을 출력하고 StopIteration 발생하는 코루틴 (yield가 1개)
1. 무한반복되는 로직 -> 제너레이터가 특정 값을 넘겨 받아서 -> 총 합을 출력해주는 코루틴


In [29]:
def a():
    total = 0
    while True:
        x = yield                 # 입력값을 처리하는 변수
        total += x                 #  입력값을 더한다 
        print(total)

In [24]:
g = a()

In [25]:
next(g)

In [26]:
g.send(100)

100


In [27]:
g.send(200)

300


In [28]:
g.send(300)

600


##  throw 메소드 정의 

- 특정 시점에 예외 발생

In [87]:
help(Generator.throw)

Help on function throw in module collections.abc:

throw(self, typ, val=None, tb=None)
    Raise an exception in the generator.
    Return next yielded value or raise StopIteration.



In [30]:
def aa():
    total = 0
    while True:
        x = yield
        total += x
        print(total)

In [31]:
gen = aa()
next(gen)

In [32]:
gen.throw(StopIteration)

RuntimeError: generator raised StopIteration

In [33]:
gen.send(12)

StopIteration: 

## close 메소드 알아보기

In [92]:
help(Generator.close)

Help on function close in module collections.abc:

close(self)
    Raise GeneratorExit inside generator.



# yield from

PEP 380 

v 3.3

In [34]:
def sub():
    while True:
        data = yield
        print("출력 data :", data)
        
        
def main():
    yield from sub()

In [35]:
co = main()

In [36]:
next(co)

In [37]:
co.send(10)

출력 data : 10


In [38]:
def sub():
    while True:
        data = yield
        print("출력 data :", data)
        
        
def main():
    x = yield from sub()
    print(x, '를 받았습니다.')

In [39]:
co = main()
next(co)

In [40]:
co.send(2222)
co.send(123)

출력 data : 2222
출력 data : 123


In [41]:
def sub():
    while True:
        data = yield
        if data is None:
            break
        print("출력 data :", data)
        
        
def main():
    x = yield from sub()
    print(x, '를 받았습니다.')

In [42]:
co = main()
next(co)

In [43]:
co.send(333)

출력 data : 333


In [44]:
co.send(None)

None 를 받았습니다.


StopIteration: 

## return  in generator 

PEP 380

In [45]:
def sub():
    while True:
        data = yield
        if data is None:
            return 12
        print("출력 data :", data)
        
        
def main():
    x = yield from sub()
    print(x, '를 받았습니다.')

In [46]:
co = main()
next(co)

In [47]:
co.send(333)

출력 data : 333


In [48]:
co.send(None)
#next(co)

12 를 받았습니다.


StopIteration: 

### 리턴받는 형태

`return value`   
`raise StopIteration(value)`

## PEP 380

하위 제너레이터에게 위임하기 위한 문법 `yield from` , `return`



```
list[i]에 1부터 (i+1)까지의 합을 가지고 있는 리스트

[1, 3, 6, ...]
```

In [49]:
def sum_all():
    total = 0
    while True:
        data = yield
        if data is None:
            break
        total += data
    return total


def recode(storage):
    while True:
        value = yield from sum_all()
        storage.append(value)


def main(n):
    results = []
    for i in range(1, n+1):
        gen = recode(results)
        next(gen)
        for j in range(i+1):
            gen.send(j)
        gen.send(None)
    return results

In [50]:
main(20)

[1,
 3,
 6,
 10,
 15,
 21,
 28,
 36,
 45,
 55,
 66,
 78,
 91,
 105,
 120,
 136,
 153,
 171,
 190,
 210]

# 반복을 반복하는 형태

In [51]:
data = {
    'tom': [100, 80, 90, 30, 40],
    'john': [30, 50, 30, 70, 100],
    'emma': [80, 39, 69, 100, 90]
}

In [52]:
def average():
    total = 0
    count = 0
    while True:
        score = yield
        if score is None:
            break
        total += score
        count += 1
    return total / count

def recode(result, name):
    while True:
        result[name] = yield from average()
        
        
def main():
    result = {}
    for name in data:
        gen = recode(result, name)
        next(gen)
        for score in data[name]:
            gen.send(score)
        gen.send(None)
    return result

In [53]:
main()

{'tom': 68.0, 'john': 56.0, 'emma': 75.6}

In [54]:
from collections.abc import Coroutine

In [55]:
dir(Coroutine)

['__abstractmethods__',
 '__await__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_abc_impl',
 'close',
 'send',
 'throw']

In [124]:
!open .